In [1]:
!pip install nba_api

In [9]:
from nba_api.stats.endpoints import scoreboardv2, boxscoretraditionalv2
from datetime import datetime, timedelta
import pandas as pd
import time

# Get dates for the past 14 days
end_date = datetime.today()
start_date = end_date - timedelta(days=14)

# Format dates for NBA API
end_str = end_date.strftime('%m/%d/%Y')

# Get today's scoreboard to retrieve recent games
scoreboard = scoreboardv2.ScoreboardV2(game_date=end_str)
games_df = scoreboard.get_data_frames()[0]
game_ids = games_df['GAME_ID'].tolist()

all_player_stats = []

# Loop through each game and collect player stats
for game_id in game_ids:
    try:
        boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
        player_df = boxscore.get_data_frames()[0]
        player_df['GAME_ID'] = game_id
        all_player_stats.append(player_df)
        time.sleep(0.6)  # Respect rate limits
    except Exception as e:
        print(f"Error with game {game_id}: {e}")

# Combine all collected data
if all_player_stats:
    full_df = pd.concat(all_player_stats, ignore_index=True)
    full_df.to_csv("player_stats_last_2_weeks.csv", index=False)
    print("✅ CSV file saved as: player_stats_last_2_weeks.csv")
else:
    print("⚠️ No data collected.")


✅ CSV file saved as: player_stats_last_2_weeks.csv


In [7]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
from datetime import datetime, timedelta

# Get today's date and 14 days before
end_date = datetime.today()
start_date = end_date - timedelta(days=14)

# Convert dates to NBA API format: 'MM/DD/YYYY'
start_date_str = start_date.strftime('%m/%d/%Y')
end_date_str = end_date.strftime('%m/%d/%Y')

# Fetch all games in the past 2 weeks
gamefinder = leaguegamefinder.LeagueGameFinder(
    date_from_nullable=start_date_str,
    date_to_nullable=end_date_str
)

# Get the results in a DataFrame
games_df = gamefinder.get_data_frames()[0]
print(games_df.columns)



Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')
